In [51]:
"""
Konventionelles Gewächshaus-Modell mit PyPSA
- Verwendet Gurobi als Optimizer
- Heizlast aus heizlast_2019.csv
- Strombedarf aus hourly_lamp_energy_2019.csv
"""

import pypsa
import pandas as pd
import numpy as np

# ============================================================
# 1. Daten einlesen
# ============================================================

# Heizlast einlesen
df_heizlast = pd.read_csv('heizlast_2019.csv', sep=',', encoding='utf-8')
df_heizlast['datetime'] = pd.to_datetime(df_heizlast['MESS_DATUM'].astype(str), format='%Y%m%d%H')
df_heizlast.set_index('datetime', inplace=True)

# Strombedarf (Lampen) einlesen
df_strombedarf = pd.read_csv('hourly_lamp_energy_2019.csv', sep=';', encoding='utf-8')
df_strombedarf['datetime'] = pd.to_datetime(df_strombedarf['DateTime'].astype(str), format='%Y%m%d%H')
df_strombedarf.set_index('datetime', inplace=True)

# ============================================================
# 2. Parameter definieren
# ============================================================

# Wärmespeicher
waermespeicher_kapazitaet = 5000            # kWh
capital_cost_waermespeicher = 10             # €/kWh/a als Annuität
waermespeicher_lifetime = 25                # Jahre
waermespeicher_standing_loss = 0.005         # Verlust pro Stunde

# Gaskessel
gaskessel_leistung = 1500                   # kW
gaskessel_wirkungsgrad = 0.95               # 95%
capital_cost_gaskessel = 7                  # €/kW/a als Annuität
gaskessel_lifetime = 20                     # Jahre

# Netzstrom
strom_preis = 0.25                          # €/kWh
gas_preis = 0.08                            # €/kWh

# ============================================================
# 3. Daten vorbereiten
# ============================================================

# Gemeinsamen Zeitindex erstellen
zeitindex = df_heizlast.index.intersection(df_strombedarf.index)

# Nur die ersten 168 Stunden (1 Woche) für schnellere Tests
# Kommentiere die nächste Zeile aus, um das ganze Jahr zu simulieren
# zeitindex = zeitindex[:168]  # 1 Woche

print(f"Simulationszeitraum: {zeitindex[0]} bis {zeitindex[-1]}")
print(f"Anzahl Zeitschritte: {len(zeitindex)}")

# Zeitreihen auf Simulationszeitraum einschränken
waermebedarf = df_heizlast.loc[zeitindex, 'Heizlast_kW']
strombedarf = df_strombedarf.loc[zeitindex, 'Energy_kW']

# Datenübersicht
print(f"\nMittlere Heizlast:      {waermebedarf.mean():.2f} kW")
print(f"Maximale Heizlast:      {waermebedarf.max():.2f} kW")
print(f"Minimale Heizlast:      {waermebedarf.min():.2f} kW")
print(f"Mittlerer Strombedarf:  {strombedarf.mean():.2f} kW")
print(f"Maximaler Strombedarf:  {strombedarf.max():.2f} kW")

# ============================================================
# 4. PyPSA-Netzwerk erstellen
# ============================================================

network = pypsa.Network()
network.set_snapshots(zeitindex)

# Busse
network.add('Bus', name='Strom', carrier='strom')
network.add('Bus', name='Waerme', carrier='waerme')
network.add('Bus', name='Gas', carrier='gas')

# Lasten
network.add('Load', name='Stromlast', bus='Strom', p_set=strombedarf)
network.add('Load', name='Waermelast', bus='Waerme', p_set=waermebedarf)

# Netzstrom (Import aus öffentlichem Netz)
network.add('Generator',
            name='Netz_Import',
            bus='Strom',
            p_nom=10000,
            marginal_cost=strom_preis,
            carrier='grid')

# Gasversorgung
network.add('Generator',
            name='Gas_Versorgung',
            bus='Gas',
            p_nom=5000,
            marginal_cost=gas_preis,
            carrier='gas')

# Wärmespeicher
network.add('Store',
            name='Waermespeicher',
            bus='Waerme',
            e_nom=waermespeicher_kapazitaet,
            e_nom_extendable=True,
            capital_cost=capital_cost_waermespeicher,
            standing_loss=waermespeicher_standing_loss,
            e_cyclic=True,
            lifetime=waermespeicher_lifetime)

# Gaskessel (Gas -> Wärme)
network.add('Link',
            name='Gaskessel',
            bus0='Gas',
            bus1='Waerme',
            p_nom=gaskessel_leistung,
            efficiency=gaskessel_wirkungsgrad,
            capital_cost=capital_cost_gaskessel,
            lifetime=gaskessel_lifetime)

# ============================================================
# 5. Optimierung mit Gurobi
# ============================================================

network.optimize(solver_name='gurobi')

# ============================================================
# 6. Ergebnisse ausgeben
# ============================================================

print("\n" + "=" * 80)
print("OPTIMIERUNGSERGEBNISSE")
print("=" * 80)



# Gesamtkosten
gesamtkosten_konventiunell = network.objective

print(f"\nGesamtkosten: {gesamtkosten_konventiunell:.2f} €")

# Strombilanz
print("\n--- Strombilanz ---")
strom_netz = network.generators_t.p['Netz_Import'].sum()
strom_last = network.loads_t.p['Stromlast'].sum()

print(f"Netzbezug:        {strom_netz:>12.2f} kWh")
print(f"Stromlast:        {strom_last:>12.2f} kWh")

# Wärmebilanz
print("\n--- Wärmebilanz ---")
waerme_kessel = network.links_t.p1['Gaskessel'].sum()
waerme_last = network.loads_t.p['Waermelast'].sum()

print(f"Gaskessel Wärme:  {waerme_kessel:>12.2f} kWh")
print(f"Wärmelast:        {waerme_last:>12.2f} kWh")

# Gasverbrauch
print("\n--- Gasverbrauch ---")
gas_kessel = network.links_t.p0['Gaskessel'].sum()

print(f"Gas Kessel:       {gas_kessel:>12.2f} kWh")

# Betriebskosten
print("\n--- Betriebskosten ---")
kosten_strom = strom_netz * strom_preis
kosten_gas = gas_kessel * gas_preis
operational_costs = round(kosten_strom + kosten_gas, 2)

print(f"Stromkosten:      {kosten_strom:>12.2f} €")
print(f"Gaskosten:        {kosten_gas:>12.2f} €")
print(f"Betriebskosten:   {operational_costs:>12.2f} €")

# Wärmespeicher
print("\n--- Wärmespeicher ---")
speicher_e = network.stores_t.e['Waermespeicher']
print(f"Durchschnitt:     {speicher_e.mean():>12.2f} kWh")
print(f"Maximum:          {speicher_e.max():>12.2f} kWh")
print(f"Minimum:          {speicher_e.min():>12.2f} kWh")

print("\n" + "=" * 80)
print("Optimierung erfolgreich abgeschlossen!")
print("=" * 80)

Simulationszeitraum: 2019-01-01 00:00:00 bis 2019-12-31 23:00:00
Anzahl Zeitschritte: 8760

Mittlere Heizlast:      381.24 kW
Maximale Heizlast:      1597.24 kW
Minimale Heizlast:      0.00 kW
Mittlerer Strombedarf:  834.01 kW
Maximaler Strombedarf:  2528.00 kW


Index(['Strom', 'Waerme', 'Gas'], dtype='object', name='name')
Index(['Netz_Import', 'Gas_Versorgung'], dtype='object', name='name')
Index(['Gaskessel'], dtype='object', name='name')
Index(['Waermespeicher'], dtype='object', name='name')
INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 6/6 [00:00<00:00, 263.10it/s]
INFO:linopy.io: Writing time: 0.27s


Set parameter ServerTimeout to value 10


INFO:gurobipy:Set parameter ServerTimeout to value 10


Set parameter TokenServer to value "139.6.183.241"


...

Solved in 479 iterations and 0.19 seconds (0.16 work units)


INFO:gurobipy:Solved in 479 iterations and 0.19 seconds (0.16 work units)


Optimal objective  2.068280034e+06


INFO:gurobipy:Optimal objective  2.068280034e+06
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 43802 primals, 105121 duals
Objective: 2.07e+06
Solver model: available
Solver message: 2

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Link-fix-p-lower, Link-fix-p-upper, Store-ext-e-lower, Store-ext-e-upper, Store-energy_balance were not assigned to the network.



OPTIMIERUNGSERGEBNISSE

Gesamtkosten: 2068280.03 €

--- Strombilanz ---
Netzbezug:          7305920.00 kWh
Stromlast:          7305920.00 kWh

--- Wärmebilanz ---
Gaskessel Wärme:   -3339684.29 kWh
Wärmelast:          3339623.66 kWh

--- Gasverbrauch ---
Gas Kessel:         3515457.15 kWh

--- Betriebskosten ---
Stromkosten:        1826480.00 €
Gaskosten:           281236.57 €
Betriebskosten:     2107716.57 €

--- Wärmespeicher ---
Durchschnitt:             1.38 kWh
Maximum:               1056.35 kWh
Minimum:                  0.00 kWh

Optimierung erfolgreich abgeschlossen!


In [ ]:
"""
Zukunftssystem Gewächshaus-Modell mit PyPSA
- Verwendet Gurobi als Optimizer
- Heizlast aus heizlast_2019.csv
- Strombedarf aus hourly_lamp_energy_2019.csv
- COP Wärmepumpe aus heatpump_cop_2019.csv
- Windkraftanlagen-Leistung aus Windanlage Leistungsdaten.csv
"""

import pypsa
import pandas as pd
import numpy as np

# ============================================================
# 1. Daten einlesen
# ============================================================

# Heizlast einlesen
df_heizlast = pd.read_csv('heizlast_2019.csv', sep=',', encoding='utf-8')
df_heizlast['datetime'] = pd.to_datetime(df_heizlast['MESS_DATUM'].astype(str), format='%Y%m%d%H')
df_heizlast.set_index('datetime', inplace=True)

# Strombedarf (Lampen) einlesen
df_strombedarf = pd.read_csv('hourly_lamp_energy_2019.csv', sep=';', encoding='utf-8')
df_strombedarf['datetime'] = pd.to_datetime(df_strombedarf['DateTime'].astype(str), format='%Y%m%d%H')
df_strombedarf.set_index('datetime', inplace=True)

# COP Wärmepumpe einlesen
df_cop = pd.read_csv('heatpump_cop_2019.csv', sep=',', encoding='utf-8')
df_cop['datetime'] = pd.to_datetime(df_cop['MESS_DATUM'].astype(str), format='%Y%m%d%H')
df_cop.set_index('datetime', inplace=True)

# Windkraftanlagen-Leistung einlesen
df_wind = pd.read_csv('Windanlage Leistungsdaten.csv', sep=';', encoding='utf-8', skiprows=4)
df_wind = df_wind[['time', 'electricity']].copy()
df_wind['datetime'] = pd.to_datetime(df_wind['time'])
df_wind.set_index('datetime', inplace=True)
df_wind = df_wind.rename(columns={'electricity': 'Wind_kW'})

# ============================================================
# 2. Parameter definieren
# ============================================================

# Windkraftanlage
capital_cost_wind = 150                     # €/kW/a als Annuität
wind_lifetime = 20                          # Jahre

# Vergleichsanlage für zeitliche Verfügbarkeit
wind_nennleistung_vergleichsanlage = 6000                    # kW - maximale Nennleistung -
wind_p_max_pu = windleistung / wind_nennleistung_vergleichsanlage
wind_p_max_pu = wind_p_max_pu.clip(lower=0, upper=1)

# Stromspeicher
capital_cost_stromspeicher = 45             # €/kWh/a als Annuität     Wert prüfen ????
stromspeicher_lifetime = 15
stromspeicher_standing_loss = 0.0001         # Verlust pro Stunde

# Wärmepumpe
capital_cost_wp = 480           # €/kW/a als Annuität
wp_lifetime = 20                # Jahre

# Wärmespeicher
capital_cost_waermespeicher = 5              # €/kWh/a als Annuität
waermespeicher_lifetime = 25                # Jahre
waermespeicher_standing_loss = 0.005         # Verlust pro Stunde

# Netzinteraktion
netz_import_kosten = 0.25       # €/kWh
netz_export_erloese = 0         # €/kWh


# ============================================================
# 3. Daten vorbereiten
# ============================================================

# Gemeinsamen Zeitindex erstellen
zeitindex = df_heizlast.index.intersection(df_strombedarf.index)
zeitindex = zeitindex.intersection(df_cop.index)
zeitindex = zeitindex.intersection(df_wind.index)

print(f"Simulationszeitraum: {zeitindex[0]} bis {zeitindex[-1]}")
print(f"Anzahl Zeitschritte: {len(zeitindex)}")

# Zeitreihen auf Simulationszeitraum einschränken
waermebedarf = df_heizlast.loc[zeitindex, 'Heizlast_kW']
strombedarf = df_strombedarf.loc[zeitindex, 'Energy_kW']
cop_zeitreihe = df_cop.loc[zeitindex, 'COP']
windleistung = df_wind.loc[zeitindex, 'Wind_kW']


# Datenübersicht
print(f"\nMittlere Heizlast:      {waermebedarf.mean():.2f} kW")
print(f"Maximale Heizlast:      {waermebedarf.max():.2f} kW")
print(f"Mittlerer Strombedarf:  {strombedarf.mean():.2f} kW")
print(f"Maximaler Strombedarf:  {strombedarf.max():.2f} kW")
print(f"Mittlerer COP:          {cop_zeitreihe.mean():.2f}")

# ============================================================
# 4. PyPSA-Netzwerk erstellen
# ============================================================

network = pypsa.Network()
network.set_snapshots(zeitindex)

# Busse
network.add('Bus', name='Strom', carrier='strom')
network.add('Bus', name='Wind', carrier='wind')
network.add('Bus', name='Waerme', carrier='waerme')

# Lasten
network.add('Load', name='Stromlast', bus='Strom', p_set=strombedarf)
network.add('Load', name='Waermelast', bus='Waerme', p_set=waermebedarf)

# Windkraftanlage -> Wind-Bus
network.add('Generator',
            name='Windkraftanlage',
            bus='Wind',
            p_nom_extendable=True,
            p_max_pu=wind_p_max_pu,
            capital_cost=capital_cost_wind,
            lifetime=wind_lifetime,
            carrier='wind')

# Netz-Export (Überschuss-Windstrom verkaufen)
network.add('Generator',
            name='Netz_Export',
            bus='Wind',
            p_nom=netz_max_export,
            marginal_cost=-netz_export_erloese,
            sign=-1,
            carrier='grid_export')

# Wind-Eigenverbrauch (Wind-Bus -> Strom-Bus)
network.add('Link',
            name='Wind_Eigenverbrauch',
            bus0='Wind',
            bus1='Strom',
            p_nom_extendable=True)    #Problem war noch auf alter nennletung von windanlage

# Speichen für Windenergie
network.add('Store',
            name = 'Stromspeicher',
            bus = 'Strom',
            e_nom_extendable = True,
            capital_cost = capital_cost_stromspeicher,
            lifetime= stromspeicher_lifetime,
            standing_loss = stromspeicher_standing_loss ,
            e_cyclic = True)


# Wärmepumpe (Strom -> Wärme) mit zeitabhängigem COP
network.add('Link',
            name='Waermepumpe',
            bus0='Strom',
            bus1='Waerme',
            efficiency=cop_zeitreihe,
            p_nom_extendable=True,
            capital_cost=capital_cost_wp,
            lifetime=wp_lifetime)

# Wärmespeicher
network.add('Store',
            name='Waermespeicher',
            bus='Waerme',
            e_nom=waermespeicher_kapazitaet,
            e_nom_extendable=True,
            capital_cost=capital_cost_waermespeicher,
            standing_loss=waermespeicher_standing_loss,
            e_cyclic=True,
            lifetime=waermespeicher_lifetime)

# Netz-Import (Backup)
network.add('Generator',
            name='Netz_Import',
            bus='Strom',
            p_nom=np.inf,
            marginal_cost=netz_import_kosten,
            carrier='grid')

# ============================================================
# 5. Optimierung mit Gurobi
# ============================================================

network.optimize(solver_name='gurobi')

# ============================================================
# 6. Ergebnisse ausgeben
# ============================================================

print("\n" + "=" * 80)
print("OPTIMIERUNGSERGEBNISSE")
print("=" * 80)


# Investitionskosten insgesamt 

invest_cost_stores      = network.stores.e_nom_opt * network.stores.capital_cost * network.stores.lifetime
invest_cost_generators  = network.generators.p_nom_opt * network.generators.capital_cost * network.generators.lifetime
invest_cost_links       = network.links.p_nom_opt * network.links.capital_cost * network.links.lifetime
df_invest_cost          = pd.concat([invest_cost_stores, invest_cost_generators, invest_cost_links]).fillna(0)
invest_cost             = round(df_invest_cost.sum(), 2)

print(f"\n--- Investitionskosten insgesamt ---")
print(df_invest_cost)
print(f"\nInvestitionskosten insgesamt: {invest_cost:.2f} €")

# Investitionskosten pro Jahr


invest_cost_year_stores      = network.stores.e_nom_opt * network.stores.capital_cost 
invest_cost_year_generators  = network.generators.p_nom_opt * network.generators.capital_cost 
invest_cost_year_links       = network.links.p_nom_opt * network.links.capital_cost 
df_invest_cost_year          = pd.concat([invest_cost_year_stores, invest_cost_year_generators, invest_cost_year_links]).fillna(0)
invest_cost_year             = round(df_invest_cost_year.sum(), 2)

print(f"\n--- Jährliche Investitionskosten ---")
print(df_invest_cost_year)
print(f"\nJährliche Investitionskosten: {invest_cost_year:.2f} €")


# Optimierte Leistung
print("\n--- Optimierte Leistung ---")
wind_opt = network.generators.p_nom_opt['Windkraftanlage']
print(f"Windanlage:       {wind_opt:>12.2f} kW ")
waermepume_opt = network.links.p_nom_opt['Waermepumpe']
print(f"Wärmepumpe:       {waermepume_opt:>12.2f} kW")

# Optimierte Speicherkapazität
stormspeicher_opt = network.stores.e_nom_opt['Stromspeicher']
print(f"Stromspeicher:       {stormspeicher_opt:>12.2f} kWh")
waermespeicher_opt = network.stores.e_nom_opt['Waermespeicher']
print(f"Wärmespeicher:       {waermespeicher_opt:>12.2f} kWh")


# Strombilanz
print("\n--- Strombilanz ---")
strom_wind_gesamt = network.generators_t.p['Windkraftanlage'].sum()
strom_wind_eigen = network.links_t.p0['Wind_Eigenverbrauch'].sum()
strom_netz_import = network.generators_t.p['Netz_Import'].sum()
strom_netz_export = network.generators_t.p['Netz_Export'].sum()
strom_wp = network.links_t.p0['Waermepumpe'].sum()
strom_last = network.loads_t.p['Stromlast'].sum()

print(f"Windkraft gesamt: {strom_wind_gesamt:>12.2f} kWh")
print(f"  Eigenverbrauch: {strom_wind_eigen:>12.2f} kWh")
print(f"  Netz Export:    {strom_netz_export:>12.2f} kWh")
print(f"Netz Import:      {strom_netz_import:>12.2f} kWh")
print(f"Wärmepumpe:       {strom_wp:>12.2f} kWh")
print(f"Stromlast:        {strom_last:>12.2f} kWh")


# Gesamtkosten Gewächshaus für ein Jahr 
print("\n--- Endkosten für Stromimport ---")
gesamt_kosten_gewaechshaus = strom_netz_import * netz_import_kosten + invest_cost_year          
print('Gesamtkosten Gewächshaus für ein Jahr:', gesamt_kosten_gewaechshaus)

'''
# Wärmebilanz
print("\n--- Wärmebilanz ---")
waerme_wp = network.links_t.p1['Waermepumpe'].sum()
waerme_last = network.loads_t.p['Waermelast'].sum()

print(f"Wärmepumpe:       {waerme_wp:>12.2f} kWh")
print(f"Wärmelast:        {waerme_last:>12.2f} kWh")

# Kostenaufschlüsselung
print("\n--- Kostenaufschlüsselung ---")
kosten_import = strom_netz_import * netz_import_kosten
erloese_export = strom_netz_export * netz_export_erloese

print(f"Stromimport:      {kosten_import:>12.2f} €")
print(f"Stromexport:     -{erloese_export:>12.2f} €")
print(f"Netto:            {kosten_import - erloese_export:>12.2f} €")

# Wärmespeicher
print("\n--- Wärmespeicher ---")
speicher_e = network.stores_t.e['Waermespeicher']
print(f"Durchschnitt:     {speicher_e.mean():>12.2f} kWh")
print(f"Maximum:          {speicher_e.max():>12.2f} kWh")
print(f"Minimum:          {speicher_e.min():>12.2f} kWh")
'''

# Kennzahlen
print("\n--- Kennzahlen ---")
strom_verbrauch = strom_wind_eigen + strom_netz_import
if strom_verbrauch > 0:
    autarkie_strom = (strom_wind_eigen / strom_verbrauch) * 100
else:
    autarkie_strom = 0.0
print(f"Stromautarkie:    {autarkie_strom:>12.2f} %")

mittlerer_cop = abs(waerme_wp / strom_wp) if strom_wp > 0 else 0
print(f"Realisierter COP: {mittlerer_cop:>12.2f}")

print("\n" + "=" * 80)
print("Optimierung erfolgreich abgeschlossen!")
print("=" * 80)


Simulationszeitraum: 2019-01-01 00:00:00 bis 2019-12-31 23:00:00
Anzahl Zeitschritte: 8760

Mittlere Heizlast:      381.24 kW
Maximale Heizlast:      1597.24 kW
Mittlerer Strombedarf:  834.01 kW
Maximaler Strombedarf:  2528.00 kW
Mittlerer COP:          6.87


Index(['Strom', 'Wind', 'Waerme'], dtype='object', name='name')
Index(['Windkraftanlage', 'Netz_Export', 'Netz_Import'], dtype='object', name='name')
Index(['Wind_Eigenverbrauch', 'Waermepumpe'], dtype='object', name='name')
Index(['Stromspeicher', 'Waermespeicher'], dtype='object', name='name')
INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 8/8 [00:00<00:00, 252.47it/s]
INFO:linopy.io: Writing time: 0.41s


Set parameter ServerTimeout to value 10


INFO:gurobipy:Set parameter ServerTimeout to value 10


Set parameter TokenServer to value "139.6.183.241"


...

Solved in 14970 iterations and 1.60 seconds (0.95 work units)


INFO:gurobipy:Solved in 14970 iterations and 1.60 seconds (0.95 work units)


Optimal objective  1.392543461e+06


INFO:gurobipy:Optimal objective  1.392543461e+06
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 78846 primals, 157685 duals
Objective: 1.39e+06
Solver model: available
Solver message: 2

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-ext-p-lower, Generator-ext-p-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, Store-energy_balance were not assigned to the network.



OPTIMIERUNGSERGEBNISSE

--- Investitionskosten insgesamt ---
name
Stromspeicher          2.930117e+06
Waermespeicher         5.736366e+05
Windkraftanlage        1.015059e+07
Netz_Export            0.000000e+00
Netz_Import            0.000000e+00
Wind_Eigenverbrauch    0.000000e+00
Waermepumpe            3.139756e+06
dtype: float64

Investitionskosten insgesamt: 16794095.53 €

--- Jährliche Investitionskosten ---
name
Stromspeicher          195341.143332
Waermespeicher          22945.462765
Windkraftanlage        507529.292647
Netz_Export                 0.000000
Netz_Import                 0.000000
Wind_Eigenverbrauch         0.000000
Waermepumpe            156987.797944
dtype: float64

Jährliche Investitionskosten: 882803.70 €

--- Optimierte Leistung ---
Windanlage:            3383.53 kW 
Wärmepumpe:             327.06 kW
Stromspeicher:            4340.91 kWh
Wärmespeicher:            4589.09 kWh

--- Strombilanz ---
Windkraft gesamt:   5801194.04 kWh
  Eigenverbrauch:   5801194.04 

In [54]:
'''
Kostenvergleich Zukunft vs. Konventiunell
'''
print(f"Gesamtkosten konventiunell pro Jahr: {gesamtkosten_konventiunell:.2f} €")
print(f'Gesamtkosten Zukunft pro Jahr: {gesamt_kosten_gewaechshaus:.2f} €')



Gesamtkosten konventiunell pro Jahr: 2068280.03 €
Gesamtkosten Zukunft pro Jahr: 1417543.46 €
